In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions

import matplotlib.pyplot as plt
import numpy as np

# Independent distributions [tensorflow_probability]

- MultivariateNormalDiag로 다변량 정규분포를 직접 만드는 방법도 있지만, Independent distribution들의 Joint distribution으로 이러한 다변량 분포를 만들어내는 방법도 있다. 
    - ```tfd.Independent(batched_normal, reinterpreted_batch_ndims=0)```로 여러개의 distribution(batch 내)들을 결합할 수 있다.

**previous example**

In [4]:
# 2-dim, single distribution

mv_normal = tfd.MultivariateNormalDiag(loc=[-1, 0.5], scale_diag=[1,1.5])
print(mv_normal)

tfp.distributions.MultivariateNormalDiag("MultivariateNormalDiag", batch_shape=[], event_shape=[2], dtype=float32)


In [7]:
# argument of 'log_prob' : 'single' vector of the 2-dimensional random variable
# output : realization of 2-dim random variable

print(mv_normal.log_prob([-0.2, 1.8]))

tf.Tensor(-2.9388978, shape=(), dtype=float32)


In [5]:
# 1-dim, 2 distributions(batch_shape=2)

batched_normal = tfd.Normal(loc=[-1, 0.5], scale=[1, 1.5])
print(batched_normal)

tfp.distributions.Normal("Normal", batch_shape=[2], event_shape=[], dtype=float32)


In [8]:
# argument of 'log_prob' : each value of a random variable of each distributions in batch.
# output : realization of 2 log_probability values of each distributions in batch

print(batched_normal.log_prob([-0.2, 1.8]))

tf.Tensor([-1.2389386 -1.699959 ], shape=(2,), dtype=float32)


**independent distribution**

- The independent distribution gives us a way to absorb some or all of the batch dimensions into the event_shape
    - For example, we could use the independent distribution to transform our batched_normal distribution so that it's equivalent to the multivariate normal diag distribution.

In [13]:
batched_normal = tfd.Normal(loc=[-1, 0.5], scale=[1, 1.5])
print(batched_normal)

# normal의 batch 속 distribution들을 independent distribution으로 변환(transform)
# args 'reinterpreted_batch_ndims' : specify how the batch dimensions should be absorbed into the event space.
### 'reinterpreted_batch_ndims=1' : there is only 1 batch dimension(2) 
independent_normal = tfd.Independent(batched_normal, reinterpreted_batch_ndims=1)
print(independent_normal)

tfp.distributions.Normal("Normal", batch_shape=[2], event_shape=[], dtype=float32)
tfp.distributions.Independent("IndependentNormal", batch_shape=[], event_shape=[2], dtype=float32)


In [14]:
print(batched_normal.log_prob([-0.2, 1.8]))

tf.Tensor([-1.2389386 -1.699959 ], shape=(2,), dtype=float32)


In [16]:
# return 'scalar' value just as we had with the multivariate normal diag distribution.

print(mv_normal.log_prob([-0.2, 1.8]))
print(independent_normal.log_prob([-0.2, 1.8]))

tf.Tensor(-2.9388978, shape=(), dtype=float32)
tf.Tensor(-2.9388976, shape=(), dtype=float32)


In [19]:
# independent log_prob = batch 1 log_prob + batch 2 log_prob

print(batched_normal.log_prob([-0.2, 1.8])[0] + batched_normal.log_prob([-0.2, 1.8])[1])

tf.Tensor(-2.9388976, shape=(), dtype=float32)


**another example of independent distribution**

In [52]:
batched_normal2 = tfd.Normal(
    loc = [[-1, 0.5], [0, 1], [0.3, -0.1]],
    scale = [[1, 1.5], [0.2, 0.8], [2, 1]]
)
print(batched_normal2)

tfp.distributions.Normal("Normal", batch_shape=[3, 2], event_shape=[], dtype=float32)


In [53]:
# 'reinterpreted_batch_ndims' default : 0
independent_normal2 = tfd.Independent(batched_normal2, reinterpreted_batch_ndims=0)
print(independent_normal2)

tfp.distributions.Independent("IndependentNormal", batch_shape=[3, 2], event_shape=[], dtype=float32)


In [66]:
print(independent_normal2.log_prob(0.5).shape)
print(independent_normal2.log_prob(0.5))

(3, 2)
tf.Tensor(
[[-2.0439386 -1.3244036]
 [-2.4345007 -0.8911075]
 [-1.6170857 -1.0989385]], shape=(3, 2), dtype=float32)


In [67]:
print(batched_normal2.log_prob(0.5).shape)
print(batched_normal2.log_prob(0.5))

(3, 2)
tf.Tensor(
[[-2.0439386 -1.3244036]
 [-2.4345007 -0.8911075]
 [-1.6170857 -1.0989385]], shape=(3, 2), dtype=float32)


In [60]:
independent_normal3 = tfd.Independent(batched_normal2, reinterpreted_batch_ndims=1)
print(independent_normal3)

tfp.distributions.Independent("IndependentNormal", batch_shape=[3], event_shape=[2], dtype=float32)


In [65]:
print(independent_normal3.log_prob(0.5).shape)
print(independent_normal3.log_prob(0.5))

(3,)
tf.Tensor([-3.3683424 -3.3256083 -2.7160242], shape=(3,), dtype=float32)


In [57]:
independent_normal4 = tfd.Independent(batched_normal2, reinterpreted_batch_ndims=2)
print(independent_normal4)

tfp.distributions.Independent("IndependentNormal", batch_shape=[], event_shape=[3, 2], dtype=float32)


In [64]:
print(independent_normal4.log_prob(0.5).shape)
print(independent_normal4.log_prob(0.5))

()
tf.Tensor(-9.409975, shape=(), dtype=float32)
